## **Data Ingestion:**



In [22]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

# Define BASE_FOLDER globally
BASE_FOLDER = "data"

def make_dir():
    """Creates required directories if they don't exist."""
    sub_dirs = ["raw", "raw/train", "raw/test"]  # Fixed missing comma
    for sub in sub_dirs:
        os.makedirs(os.path.join(BASE_FOLDER, sub), exist_ok=True)

def load_data(url):
    """Loads data from the provided URL."""
    return pd.read_csv(url)

def processing(df):
    """Filters, maps sentiment values, and shuffles data."""
    return (
        df[df["sentiment"].isin(["happiness", "sadness"])]
        .drop(columns=["tweet_id"], errors="ignore")  # Avoid KeyError if column is missing
        .assign(sentiment=lambda x: x["sentiment"].map({"happiness": 1, "sadness": 0}))
        .sample(frac=1, random_state=42)  # Shuffle the data
    )

def save_data(final_df):
    """Splits data into train and test sets and saves them."""
    train_df, test_df = train_test_split(final_df, test_size=0.2, random_state=42)

    train_df.to_csv(os.path.join(BASE_FOLDER, "raw/train", "train.csv"), index=False)
    test_df.to_csv(os.path.join(BASE_FOLDER, "raw/test", "test.csv"), index=False)

def main():
    make_dir()
    url = "https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv"

    df = load_data(url)
    final_df = processing(df)
    save_data(final_df)

if __name__ == "__main__":
    main()


## **Data cleaning:**

In [23]:
import re
# NLTK for natural language processing
import nltk
from nltk.corpus import stopwords    # For stopwords
from nltk.tokenize import word_tokenize # For tokenization
from nltk.stem import PorterStemmer, WordNetLemmatizer # For stemming and lemmatization

# Downloading NLTK data
nltk.download('stopwords')   # Downloading stopwords data
nltk.download('punkt')       # Downloading tokenizer data
nltk.download('wordnet')     # Downloading WordNet data for lemmatization

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [24]:


def data_cleaning(text_series):
    """Cleans the text data by removing URLs, emails, numbers, and punctuation."""
    number_pattern = r"(?<=\D)\d+|\d+(?=\D)"  # Removes numbers but keeps letters
    url_pattern = r"https?://\S+|www\.\S+"
    email_pattern = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b"
    punctuation_pattern = r"[^\w\s]"

    return (
        text_series.astype(str)  # Ensure text is string
        .str.lower()
        .str.replace(url_pattern, " ", regex=True)
        .str.replace(email_pattern, " ", regex=True)
        .str.replace(number_pattern, " ", regex=True)
        .str.replace(punctuation_pattern, " ", regex=True)
        .str.strip()
        .str.replace(r"\s+", " ", regex=True)  # Normalize spaces
    )

def remove_short_words(text_series, min_length=3):
    """Removes words shorter than `min_length` characters."""
    return text_series.apply(lambda x: " ".join([word for word in x.split() if len(word) >= min_length]))

def lemmatization(text_series):
    """Lemmatizes words using WordNetLemmatizer."""
    lemmatizer = WordNetLemmatizer()
    return text_series.apply(lambda x: " ".join([lemmatizer.lemmatize(word, pos="v") for word in x.split()]))

def remove_stopwords(text_series):
    """Removes stopwords from text."""
    stop_words = frozenset(stopwords.words("english"))  # Faster lookup
    return text_series.apply(lambda x: " ".join([word for word in x.split() if word not in stop_words]))

def normalize(df):
    """Applies text preprocessing steps."""
    df["content"] = data_cleaning(df["content"])
    df["content"] = remove_short_words(df["content"])
    df["content"] = lemmatization(df["content"])
    df["content"] = remove_stopwords(df["content"])
    return df

def main():
    train_data = pd.read_csv("/content/data/raw/train/train.csv")
    test_data = pd.read_csv("/content/data/raw/test/test.csv")

    # Transform the data
    train_processed_data = normalize(train_data)
    test_processed_data = normalize(test_data)

    # Store the data inside data/processed
    data_path = os.path.join("./data", "interim")
    os.makedirs(data_path, exist_ok=True)

    train_processed_data.to_csv(os.path.join(data_path, "train_processed.csv"), index=False)
    test_processed_data.to_csv(os.path.join(data_path, "test_processed.csv"), index=False)


if __name__ == "__main__":
    main()


## **Feature Engineering:**

In [25]:
import pickle
import pandas as pd
import os
import logging
import yaml
from sklearn.feature_extraction.text import CountVectorizer

# Set up logger
logger = logging.getLogger("feature_engineering")
logger.setLevel(logging.DEBUG)

# Remove existing handlers to avoid duplicates
if logger.hasHandlers():
    logger.handlers.clear()

# Define log file path
os.makedirs("logs", exist_ok=True)
log_file_path = "logs/feature_engineering.log"

# Delete the log file if it exists
if os.path.exists(log_file_path):
    os.remove(log_file_path)

# FileHandler to log messages to a file
file_handler = logging.FileHandler(log_file_path)
file_handler.setLevel(logging.DEBUG)

# ConsoleHandler to log messages to the console
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.DEBUG)

# Create a formatter and attach it to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)
console_handler.setFormatter(formatter)

# Add handlers to the logger
logger.addHandler(file_handler)
logger.addHandler(console_handler)

def load_params(params_path: str) -> dict:
    """Load parameters from a YAML file."""
    try:
        with open(params_path, "r") as file:
            params = yaml.safe_load(file)
            if params is None:
                raise ValueError(f"YAML file {params_path} is empty or invalid.")
        logger.debug("Parameters retrieved from %s", params_path)
        return params
    except FileNotFoundError:
        logger.error("File not found: %s", params_path)
        raise
    except yaml.YAMLError as e:
        logger.error("YAML error: %s", e)
        raise
    except Exception as e:
        logger.error("Unexpected error: %s", e)
        raise

def load_data(path: str) -> pd.DataFrame:
    """Load data from a CSV file."""
    try:
        df = pd.read_csv(path)
        df.fillna("", inplace=True)
        logger.debug("Data loaded successfully from %s", path)
        return df
    except FileNotFoundError:
        logger.error("File not found: %s", path)
        raise
    except Exception as e:
        logger.error("Error loading file %s: %s", path, e)
        raise

def bow(train: pd.DataFrame, test: pd.DataFrame, max_feature: int):
    """Perform Bag-of-Words transformation."""
    try:
        x_train = train["content"].values
        y_train = train["sentiment"].values
        x_test = test["content"].values
        y_test = test["sentiment"].values

        vectorizer = CountVectorizer(max_features=max_feature)
        x_train_bow = vectorizer.fit_transform(x_train)
        x_test_bow = vectorizer.transform(x_test)

        train_df = pd.DataFrame(x_train_bow.toarray())
        train_df["label"] = y_train
        test_df = pd.DataFrame(x_test_bow.toarray())
        test_df["label"] = y_test

        # Ensure the models directory exists
        os.makedirs("models", exist_ok=True)
        pickle.dump(vectorizer, open("models/vectorizer.pkl", "wb"))
        logger.debug("Vectorizer saved successfully to models/vectorizer.pkl")

        return train_df, test_df
    except Exception as e:
        logger.error("Error in Bag-of-Words transformation: %s", e)
        raise

def main():
    try:
        logger.info("Starting feature engineering process.")
        params = load_params("params.yaml")
        max_features = params["feature_engineering"]["max_features"]
    except Exception as e:
        logger.error("Error loading parameters: %s", e)
        return

    try:
        train = load_data(os.path.join("/content", "data", "interim", "train_processed.csv"))
        test = load_data(os.path.join("/content", "data", "interim", "test_processed.csv"))
    except Exception as e:
        logger.error("Error loading data: %s", e)
        return

    try:
        train_df, test_df = bow(train, test, max_features)
        logger.info("Bag-of-Words transformation completed successfully.")
        print("Training DataFrame:")
        print(train_df.head())  # Replace display with print for standalone scripts
        print("\nTesting DataFrame:")
        print(test_df.head())  # Replace display with print for standalone scripts
    except Exception as e:
        logger.error("Error in Bag-of-Words transformation: %s", e)
        return

if __name__ == "__main__":
    main()


2025-02-22 00:13:11,146 - feature_engineering - INFO - Starting feature engineering process.
INFO:feature_engineering:Starting feature engineering process.
2025-02-22 00:13:11,155 - feature_engineering - DEBUG - Parameters retrieved from params.yaml
DEBUG:feature_engineering:Parameters retrieved from params.yaml
2025-02-22 00:13:11,169 - feature_engineering - DEBUG - Data loaded successfully from /content/data/interim/train_processed.csv
DEBUG:feature_engineering:Data loaded successfully from /content/data/interim/train_processed.csv
2025-02-22 00:13:11,176 - feature_engineering - DEBUG - Data loaded successfully from /content/data/interim/test_processed.csv
DEBUG:feature_engineering:Data loaded successfully from /content/data/interim/test_processed.csv
2025-02-22 00:13:11,480 - feature_engineering - DEBUG - Vectorizer saved successfully to models/vectorizer.pkl
DEBUG:feature_engineering:Vectorizer saved successfully to models/vectorizer.pkl
2025-02-22 00:13:11,483 - feature_engineerin

Training DataFrame:
   0  1  2  3  4  5  6  7  8  9  ...  4991  4992  4993  4994  4995  4996  \
0  0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   
1  0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   
2  0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   
3  0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   
4  0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   

   4997  4998  4999  label  
0     0     0     0      1  
1     0     0     0      1  
2     0     0     0      1  
3     0     0     0      1  
4     0     0     0      0  

[5 rows x 5001 columns]

Testing DataFrame:
   0  1  2  3  4  5  6  7  8  9  ...  4991  4992  4993  4994  4995  4996  \
0  0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   
1  0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   
2  0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   


In [26]:
train= pd.read_csv("/content/data/interim/train_processed.csv")
test=pd.read_csv("/content/data/interim/test_processed.csv")
train.fillna('', inplace=True)
test.fillna('', inplace=True)

In [27]:
x_train= train["content"].values
y_train= train["sentiment"].values
x_test= test["content"].values
y_test= test["sentiment"].values

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
vectorize=CountVectorizer(max_features=500)
x_train_bow = vectorize.fit_transform(x_train,)
x_test_bow = vectorize.transform(x_test)

In [29]:
train_df = pd.DataFrame(x_train_bow.toarray())
train_df['label'] = y_train
test_df = pd.DataFrame(x_test_bow.toarray())
test_df['label'] = y_test